In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

22/01/09 22:14:12 WARN Utils: Your hostname, wshid-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
22/01/09 22:14:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 22:14:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
directory = "/Users/sion/Workspace/data-engineering/01-spark/data"
filename = "fhvhv_tripdata_2020-03.csv"

In [4]:
data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)

In [5]:
data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [6]:
data.createOrReplaceTempView("mobility_data")

In [7]:
spark.sql("select * from mobility_data limit 5").show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [8]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-06|872012|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-16|391518|
| 2020-03-20|261900|
| 2020-03-19|252773|
| 2020-03-17|312298|
| 2020-03-18|269232|
+-----------+------+
only showing top 20 rows



In [9]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#127]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#126]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#126], [pickup_date#126, count(1) AS trips#127L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(pickup_datetime#18,  , -1)[0] AS pickup_date#126]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22])
            +- Relation [hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#126], [pickup_date#126, count(1) AS trips#127L]
+- Project [split(pick